In [1]:
%load_ext autoreload
%autoreload 2

from hyper_qa.model import HyperQA
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()

C:\Users\nelson.yao\AppData\Roaming\Python\Python36\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.random.set_random_seed(212)

In [3]:
embedding_matrix = np.random.normal(0, 1, (1000, 300))
def generate_samples(max_length, sample_size, vocab_size=1000, pad=False):
    lengths = []
    sample_sequences = []
    for i in range(sample_size):
        text = np.random.randint(1,1000, np.random.randint(1,max_length))
#         text = np.arange(i%max_length)
        if pad:
            sample = np.concatenate((text, [0] * (max_length - len(text))))
        else:
            sample = text
        lengths.append(len(text))
        sample_sequences.append(sample)
    if pad:
        sample_sequences = np.vstack(sample_sequences)
    return sample_sequences

In [4]:
max_length = 50
q = generate_samples(max_length, 200, pad=True)
a = generate_samples(max_length, 200, pad=True)
b = generate_samples(max_length, 200, pad=True)

In [5]:
dataset = tf.data.Dataset.from_tensor_slices((q, a, b))
dataset = dataset.shuffle(200).batch(5)
data_iter = dataset.make_one_shot_iterator()

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
q, a, b = data_iter.get_next()
model = HyperQA(1000, max_length, embedding_matrix=embedding_matrix)
logits = model((q, a, b), training=True)

In [7]:
margin = tf.constant(5.0)
loss_history = []
with tf.GradientTape() as tape:
    sim_pos, sim_neg = model((q,a,b), training=True)
    loss = tf.reduce_mean(tf.maximum(0.0, margin + sim_neg - sim_pos))

gradients = tape.gradient(loss, model.trainable_variables)

In [8]:
for param in [model.bow_q1, model.bow_q2, model.bow_q3]:
    print(tf.norm(param, axis=-1).numpy())
print(model.distance_neg)

[0.99999994 1.         1.         1.0000002  0.99999994]
[1.0000002  0.99999976 0.99999994 1.0000001  1.0000005 ]
[1.0000002 1.        1.0000002 1.0000001 1.0000001]
tf.Tensor([ 9.567393  8.662656  9.297244 10.082652 10.720236], shape=(5,), dtype=float32)
